# 第11回課題

In [19]:
# このセルは決して書き換えたり消したりしないこと
import os
from pytest import approx

def local_run(t):
    """
    自動採点を妨げずにテストを実行するための補助関数。
    """
    if not ('GRADING' in os.environ):
        t()

## Q1. 正方行列の固有ベクトル

$n\times n$の行列(正方行列)$A$、初期ベクトル$x_0 \in R^n$、十分小さい量を判定するための実数$\epsilon$が与えられた時、べき乗法により$A$の固有ベクトルを求める関数`eigenvector()`を定義せよ。ただし、
- 行列は行をリストとするリスト(2重リスト)で表されるものとする。
- $\epsilon$の初期値は$0.001$とせよ。

この関数は全ての固有ベクトルを求めるものではなく、最大の絶対値を持つ固有値に属する固有ベクトルを求めるものである。

### 説明: べき乗法

べき乗法は、$n \times n$行列の最大の絶対値を持つ固有値とその固有ベクトルを求めるための反復的な手法である。以下の手順で計算を行う。

1. **初期ベクトルの選択**: 適当な初期ベクトル $$x^{(0)}$$ を選ぶ。通常、すべての要素が1のベクトルを用いる。
2. **行列の乗算**: 行列 $A$ とベクトル $x^{(k)}$ を乗算する: $$y^{(k)} = Ax^{(k)}$$
3. **スケーリング**: $y^{(k)}$ のノルムで $y^{(k)}$ をスケーリングする: $$x^{(k+1)} = \frac{y^{(k)}}{\lVert y^{(k)}\rVert}$$
$\lVert v \rVert$はベクトル$v$のノルムを表す。
4. **収束チェック**: $$\lVert x^{(k+1)} - x^{(k)}\rVert < \epsilon $$ になるまで、手順2と3を繰り返す。


べき乗法が収束したら、$x^{(k+1)}$が固有ベクトルになっている。

参考:
- Wikipedia: https://ja.wikipedia.org/wiki/%E3%81%B9%E3%81%8D%E4%B9%97%E6%B3%95

### Step 1. テストケースを考え、`eigenvertor()`のテストを完成させよ

In [22]:
def test_eigenvertor():
    """
    eigenvector()をテストする
    """
    # [[3,1],[3,1]]の最大絶対値の固有値は4であり、その固有ベクトルはc*[1,1](cは任意の実数)である。
    # コンピュータによる計算のため、厳密に1になることはない。
    # したがってapprox()を使い、1にとても近いことを確かめる。
    # epsilonは初期値を使う
    u = eigenvector([[3,1],[3,1]], [1,1])
    assert u[1]/u[0] == approx(1)  # 第1成分と第2成分の比が1

    # 以下にテストケースを追加せよ
    u = eigenvector([[3, 1], [3, 1]], [1, 1])
    assert u[1] / u[0] == approx(1)  # 第1成分と第2成分の比が1に近い

### Step 2. `eigenvector()`を完成させよ

In [24]:
def eigenvector(A, x0, epsilon=0.001):
    """
    べき乗法により行列Aの固有ベクトルを求める

    Parameters:
        A (list): 行を表すリストのリスト(2重リスト)で表した行列
        x0 (list): 初期ベクトル
        epsilon (float): 十分小さい値を表す実数(初期値は0.001)

    Returns:
        list: 行列Aの固有ベクトルを表すリスト
    """
    x = x0[:] # 初期ベクトル x0 をコピーして x に格納する
    while True:
        x_next = [0] * len(x) # 次のベクトル x_next を初期化する
        # 行列 A とベクトル x の積を計算する
        for i in range(len(A)):
            for j in range(len(x)):
                x_next[i] += A[i][j] * x[j]

        # ベクトルのノルムを計算
        norm = sum(x ** 2 for x in x_next) ** 0.5

        # ベクトルを正規化
        x_next = [x / norm for x in x_next]

        # 収束判定
        if sum(abs(x_next[i] - x[i]) for i in range(len(x))) < epsilon:
            break

        x = x_next

    return x

### Step 3. テストを実行せよ

In [23]:
local_run(test_eigenvertor)

## Q2. Minesweeper

Minesweeperは一人用のコンピュータゲームである。ゲームの目的は、地雷が埋められていると予想される場所にフラグを立て、地雷がない場所をクリックして開けることである。プレイヤーがすべての地雷の位置を正しく特定し、地雷がないすべての場所を開けたとき、プレイヤーの勝利となる。

- Wikipedia: https://ja.wikipedia.org/wiki/%E3%83%9E%E3%82%A4%E3%83%B3%E3%82%B9%E3%82%A4%E3%83%BC%E3%83%91

- 遊ぶ: https://minesweeper.online/ja/

あなたの任務は、地雷(M)の設置されたボード($n\times n$)が与えられた時、地雷以外のマスを隣接地雷数で埋めたボードを作成することである。例えば、以下のボードが与えられた時、

|     | M  |     |
|-----|---|-----|
|     |   | M |
|     |   |  |

以下のボードを作成する。

| 1    | M  |  2   |
|-----|---|-----|
|  1   | 2  | M |
|     |  1 | 1 |

ボードは文字列を要素とするリストのリスト(2重リスト)で表し、地雷はMineの頭文字である'M'で表すものとする。地雷の設置されたボードを受け取り、地雷以外のマスを隣接地雷数(ただし数値ではなく文字列)で埋めたボードを作成する関数`fill_board()`を完成させよ。例えばこの関数は、

```python
[
    ['', 'M', ''],
    ['', '', 'M'],
    ['', '', ''],
]
```

を入力としたとき、

```python
[
    ['1', 'M', '2'],
    ['1', '2', 'M'],
    ['0', '1', '1'],
]
```

を返す。

#### Step 1. テストケースを考え、`test_fill_board()`を完成させよ

In [29]:
def test_fill_board():
    """
    fill_board()をテストする
    """
    input_board = [
        ['', 'M', ''],
        ['', '', 'M'],
        ['', '', ''],
    ]

    expected = [
        ['1', 'M', '2'],
        ['1', '2', 'M'],
        ['0', '1', '1'],
    ]
    actual = fill_board(input_board)
    assert actual == expected

    # 以下にテストケースを追加せよ

    # 全てのセルに地雷がある場合
    input_board_all_mines = [
        ['M', 'M', 'M'],
        ['M', 'M', 'M'],
        ['M', 'M', 'M'],
    ]
    expected_all_mines = [
        ['M', 'M', 'M'],
        ['M', 'M', 'M'],
        ['M', 'M', 'M'],
    ]
    assert fill_board(input_board_all_mines) == expected_all_mines

#### Step 2. `fill_board()`を完成させよ

In [25]:
def fill_board(board):
    """
    地雷の設置されたボードを受け取り、隣接地雷数で埋めたボードを返す

    Parameters:
        board (list): 地雷の設置されたボード

    Returns:
        list: 隣接地雷数で埋めたボード
    """
     # ボードのサイズを取得する
    n = len(board)

    # 方向ベクトル(上下左右と斜め)
    directions = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),         (0, 1),
                  (1, -1), (1, 0), (1, 1)]

    # 隣接地雷数で埋めたボードを初期化する
    filled_board = [[''] * n for _ in range(n)]

    # ボードの各セルをチェックして隣接地雷数を計算する
    for i in range(n):
        for j in range(n):
            if board[i][j] == 'M':
                filled_board[i][j] = 'M'
            else:
                # 周囲の地雷数を数える
                mine_count = 0
                for di, dj in directions:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < n and 0 <= nj < n and board[ni][nj] == 'M':
                        mine_count += 1
                # 数字を文字列に変換して埋める
                filled_board[i][j] = str(mine_count)

    return filled_board

### Step 3. テストを実行せよ

In [30]:
local_run(test_fill_board)

### おまけ: Minesweeperで遊ぶ

一回クリックすると赤(flagが立てられた状態)になり、二回クリックすると開く。Shiftキーを押しながらクリックすると、flagを取り消すことができる。

注意: colab上で無理矢理画面表示しているため、マスの色の切り替わりに時間がかかる。また、以下の機能はない。
- 発見した地雷数のカウント
- 地雷を踏んだときにゲーム終了

In [31]:
# 必要ファイルのダウンロード
!pip install solara
!wget --no-check-certificate -O minesweeper.7z "https://docs.google.com/uc?export=download&id=1BT8KzkLlEVUfbgzALyOdYBYHw4gMrnaZ"
!7z x minesweeper.7z

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [32]:
# ゲームの実行(再実行もこのセルだけでよい)
BOARD_SIZE = 10   # 10x10マスのボード
NUM_OF_MINES = 10 # 地雷の数
def run_game():
    import minesweeper
    minesweeper.game(BOARD_SIZE, NUM_OF_MINES, fill_board)

local_run(run_game)

Html(layout=None, style_='display: none', tag='span')

<IPython.core.display.Javascript object>

Cannot show ipywidgets in text